In [11]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

import pickle
import torch
import importlib
import numpy as np
import pandas as pd

import src.bin.tensorify as tensorify
import src.utils.data_conversion_utils as conversions
import src.data_manager.student_life_var_binned_data_manager as data_manager
import src.bin.trainer as trainer

from sklearn import metrics

from torch import nn
from src import definitions
from src.bin import statistics
from src.bin import plotting
from src.utils.read_utils import read_pickle
from src.utils import student_utils
from tabulate import tabulate 
import src.models.simple as simple_models   

pd.set_option('max_columns', 10000)

importlib.reload(statistics)
importlib.reload(conversions)
importlib.reload(tensorify)
importlib.reload(plotting)
importlib.reload(trainer)
importlib.reload(data_manager)
importlib.reload(student_utils)
importlib.reload(simple_models)
feature_list = data_manager.FEATURE_LIST

student_list = [53, 46, 7, 49, 24, 22, 2]
student_list = [2]

data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, fill_na=True)



########### Stats ########### 

stats_df, raw_data = statistics.get_statistics_on_data_dict(data, feature_list)
print(statistics.get_train_test_val_label_counts_from_raw_data(data))
display(stats_df)







# csv_data = []
# for key in data['test_ids']:
#     csv_data.append(data['data'][key][0] + [data['data'][key][3]])
    
# print(len(csv_data))
    
# df = pd.DataFrame(csv_data)
# df.to_csv('test_random_forest_traning_data.csv')





1728
      Train    Val    Test
--  -------  -----  ------
 0        4      3       0
 1        8      3       1
 2        2      2       1
       day_of_week  epoch_of_day  time_since_last_label  time_to_next_label  \
count  1728.000000   1728.000000            1728.000000         1728.000000   
mean      2.784722      1.597222            1472.829861          463.402778   
std       1.875684      1.309107            1251.777208          911.545832   
min       0.000000      0.000000               0.000000            0.000000   
25%       1.000000      0.000000             538.750000           85.000000   
50%       3.000000      2.000000            1205.000000          175.000000   
75%       4.000000      3.000000            1676.250000          265.000000   
max       6.000000      3.000000            4435.000000         4675.000000   

       activity_inference_inferred_feature  \
count                          1728.000000   
mean                              0.423611   
std       

In [37]:
def get_training_data_and_labels(data, split):
    assert split in ['train', 'val', 'test'], "Invalid split enetered."
    
    training_data, labels = [], []
    
    for key in data[split+"_ids"]:
        training_data.append(data['data'][key][0])
        labels.append(data['data'][key][3])
        
    return training_data, labels

train_x, train_y = get_training_data_and_labels(data, 'train')
val_x, val_y = get_training_data_and_labels(data, 'val')
test_x, test_y = get_training_data_and_labels(data, 'test')

from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=20, criterion='gini')

random_forest.fit(train_x, train_y)

train_preds = random_forest.predict(train_x)
val_preds = random_forest.predict(val_x)
test_preds = random_forest.predict(test_x)

train_scores = metrics.precision_recall_fscore_support(train_y, train_preds, average='macro')
val_scores = metrics.precision_recall_fscore_support(val_y, val_preds, average='weighted')
test_scores = metrics.precision_recall_fscore_support(test_y, test_preds, average='weighted')

print("Train F1: {} Val F1: {} Test F1: {}".format(train_scores[2], val_scores[2], test_scores[2]))




Train F1: 1.0 Val F1: 0.40524499654934437 Test F1: 0.5831168831168831
